In [1]:
import os
import pandas as pd
from geo import get_place_list, get_place_data
from params import DATA_DIR, SRC_DATA_DIR

Load some data

* `all_geos` is a list of all geographies mentioned in the the geography tree csv file
* `population_data` is population estimates for a (some of) the geographies
* `council_tax_data` is estimates of people in receipt of council tax support
* `clif_data` is data bout children living in poverty 

In [2]:
population_data = pd.read_csv(f'{DATA_DIR}/population-estimates/population-estimates.csv')
population_data = population_data[population_data.age_name == "All Ages"]
group = population_data
#population_data = population_data[population_data.date == max(population_data['date'])]
population_data = population_data.loc[population_data.groupby('geography_code')['date'].idxmax()]
#population_data = population_data.query()
#print(group)
population_data

,date,geography_code,geography_name,geography_type,gender_code,gender_name,age_code,age_name,variable_code,variable_name,value
87696,2020,E05000026,Abbey,wd22,0,Total,200,All Ages,n_persons,Number of persons,16149
87714,2020,E05000027,Alibon,wd22,0,Total,200,All Ages,n_persons,Number of persons,10907
87732,2020,E05000028,Becontree,wd22,0,Total,200,All Ages,n_persons,Number of persons,15182
87750,2020,E05000029,Chadwell Heath,wd22,0,Total,200,All Ages,n_persons,Number of persons,11463
87768,2020,E05000030,Eastbrook,wd22,0,Total,200,All Ages,n_persons,Number of persons,11557
...,...,...,...,...,...,...,...,...,...,...,...
6264,2021,E47000007,West Midlands,cauth22,0,Total,200,All Ages,n_persons,Number of persons,2916132
6282,2021,E47000008,Cambridgeshire and Peterborough,cauth22,0,Total,200,All Ages,n_persons,Number of persons,896756
6300,2021,E47000009,West of England,cauth22,0,Total,200,All Ages,n_persons,Number of persons,954276
6318,2021,E47000010,North East,cauth22,0,Total,200,All Ages,n_persons,Number of persons,1139626


In [3]:
gva = pd.read_csv(f'{DATA_DIR}/gva/gva.csv')
#gva['date'].astype(float)
#gva[gva['geography_code'].str.startswith('E09')]
gva['date'] = pd.to_datetime(gva['date'])
gva['date'] = gva['date'].dt.year
gva = gva.loc[gva.groupby('geography_code')['date'].idxmax()]
gva[gva['geography_code'].str.startswith('E05')]

,date,geography_code,variable_name,variable_unit,value
172260,2020,E05000026,GVA,£m,487.120255
172261,2020,E05000027,GVA,£m,118.668199
172262,2020,E05000028,GVA,£m,80.469218
172263,2020,E05000029,GVA,£m,107.323509
172264,2020,E05000030,GVA,£m,201.595050
...,...,...,...,...,...
179242,2020,E05013859,GVA,£m,108.915558
179243,2020,E05013860,GVA,£m,82.712088
179244,2020,E05013861,GVA,£m,189.998642
179245,2020,E05013862,GVA,£m,58.629043


In [4]:
area_of_place = pd.read_csv(f'{DATA_DIR}/geo/area_of_places.csv')
#@TODO temporary filter to remove
area_of_place.drop_duplicates(subset=['geography_code'], inplace=True)
# len(area_of_place.geography_code)
# l1 = area_of_place.geography_code.to_list()
# l2 = area_of_place.geography_code.unique()
# seen = set()
# dupes = [x for x in l1 if x in seen or seen.add(x)]
# print(dupes)
area_of_place

,geography_code,variable_name,value
0,E05000650,Area in sq km,6.556496
1,E05000651,Area in sq km,8.998946
2,E05000652,Area in sq km,3.719582
3,E05000653,Area in sq km,7.340085
4,E05000654,Area in sq km,3.497711
...,...,...,...
1646,E11000007,Area in sq km,538.992682
1647,E12000001,Area in sq km,8563.803423
1648,E12000002,Area in sq km,14103.955722
1649,E12000003,Area in sq km,15369.599209


In [5]:
households = pd.read_csv(f'{DATA_DIR}/households/households.csv')
households = households.loc[households.groupby('geography_code')['date'].idxmax()]
households

,date,geography_code,variable_name,value
0,2022,E05000026,Number of households,5800
1,2022,E05000027,Number of households,4090
2,2022,E05000028,Number of households,5380
3,2022,E05000029,Number of households,4300
4,2022,E05000030,Number of households,4100
...,...,...,...,...
7825,2022,W05001034,Number of households,940
7826,2022,W05001035,Number of households,2310
7827,2022,W05001036,Number of households,1910
7828,2022,W05001037,Number of households,860


In [6]:
council_tax_data = pd.read_csv(f'{DATA_DIR}/council-tax-support/council-tax-support.csv')
#council_tax_data = council_tax_data[council_tax_data.date == max(council_tax_data.date)]
council_tax_data['date'] = pd.to_datetime(council_tax_data['date'])
council_tax_data['date'] = council_tax_data['date'].dt.year
council_tax_data = council_tax_data.loc[council_tax_data.groupby(['geography_code', 'variable_name'])['date'].idxmax()]
council_tax_data.replace("pensioners", "council_tax_pensioners", inplace=True)
council_tax_data.replace("working_age", "council_tax_working_age", inplace=True)
council_tax_data[council_tax_data.variable_name == "council_tax_working_age"]
#council_tax_data[council_tax_data.geography_code.str.startswith('E05')]

,date,geography_code.ba,geography_code,geography_name,geography_type,variable_code,variable_name,value
14876,2022,E0701,E06000001,Hartlepool,UA,council_tax_working_age,council_tax_working_age,8712.0
16302,2022,E0702,E06000002,Middlesbrough,UA,council_tax_working_age,council_tax_working_age,12738.0
17480,2022,E0703,E06000003,Redcar & Cleveland,UA,council_tax_working_age,council_tax_working_age,8028.0
19185,2022,E0704,E06000004,Stockton-on-Tees,UA,council_tax_working_age,council_tax_working_age,11143.0
13202,2022,E1301,E06000005,Darlington,UA,council_tax_working_age,council_tax_working_age,6044.0
...,...,...,...,...,...,...,...,...
10691,2022,NaN,E12000006,East of England,NaN,council_tax_working_age,council_tax_working_age,211975.0
10722,2022,NaN,E12000007,London,NaN,council_tax_working_age,council_tax_working_age,445550.0
10815,2022,NaN,E12000008,South East England,NaN,council_tax_working_age,council_tax_working_age,284443.0
10846,2022,NaN,E12000009,South West England,NaN,council_tax_working_age,council_tax_working_age,207054.0


In [7]:
clif_data = pd.read_csv(f'{DATA_DIR}/clif/clif_REL.csv')
clif_data = clif_data[
    (clif_data['Age of Child (years and bands)'] == 'Total') &
    (clif_data['Gender of Child'] == 'Total') &
    (clif_data['Family Type'] == 'Total') &
    (clif_data['Work Status'] == 'Total') &
    (clif_data.Year == max(clif_data.Year))
]
clif_data[clif_data.geography_code == 'E08000035']

,Year,Age of Child (years and bands),Gender of Child,Family Type,Work Status,variable_name,geography_code,value
11474,2021/22,Total,Total,Total,Total,children_in_low_income,E08000035,39995.0


In [8]:
number_of_persons = pd.read_csv(f'{DATA_DIR}/population-estimates/population-estimates-ages.csv')
number_of_children = number_of_persons[number_of_persons.age_band.isin(['0-4', '5-10', '11-15', '16-19'])]
number_of_children = number_of_children.groupby('geography_code').sum(numeric_only=True).reset_index()
number_of_children['variable_name'] = 'number_of_children'
number_of_children

,geography_code,date,value,variable_name
0,E05000650,8080,3390,number_of_children
1,E05000651,8080,2530,number_of_children
2,E05000652,8080,3722,number_of_children
3,E05000653,8080,2813,number_of_children
4,E05000654,8080,4875,number_of_children
...,...,...,...,...
1315,E47000003,8084,586078,number_of_children
1316,E47000004,8084,346827,number_of_children
1317,E47000006,8084,159599,number_of_children
1318,E47000010,8084,247569,number_of_children


In [9]:
#number of households in poverty
households_in_poverty = pd.read_csv(f'{DATA_DIR}/hbai/Type of Individual by Age Category.csv')
households_in_poverty = households_in_poverty[households_in_poverty.variable_name == "In low income (below threshold)"]

def three_year_average(data):
    #@TODO function will need work to account for 3 year average of any period given.
    #filtering out 2020-21 as no data was collected
    data = data[data["Financial Year"] != "2020-21"]
    #getting a list of dates
    dates = data["Financial Year"].unique()
    most_recent_dates = list(dates[-2:])
    #print(most_recent)
    data = data[data["Financial Year"].isin(most_recent_dates)]
    #calculating a 2 year average according to user guidance
    data = data.groupby('geography_code').sum(numeric_only=True) / len(most_recent_dates)
    #rounding to nearest 0.1mil, according to user guidance.
    data = data.round(-5).reset_index()
    return data

households_in_poverty = three_year_average(households_in_poverty)

households_in_poverty['variable_name'] = 'Households in poverty'
households_in_poverty

,geography_code,value,variable_name
0,E12000001,500000.0,Households in poverty
1,E12000002,1300000.0,Households in poverty
2,E12000003,1100000.0,Households in poverty


In [10]:
savings = pd.read_csv(f'{DATA_DIR}/hbai/Savings and Investments of Adults in the Family of the Individual.csv')
savings = savings[savings['Financial Year'] == max(savings['Financial Year'])]
savings = savings[savings.variable_name == 'In low income (below threshold)']
savings = savings[savings['Savings and Investments of Adults in the Family of the Individual'] == 'No savings']
savings = three_year_average(savings)
savings['variable_name'] = 'households_low_income_no_savings'
savings
#save each geog_code into 'src/place/EXXXXXX/_data/' and then can build the visualisations in place.njk on a conidtional statement if that data exists.

,geography_code,value,variable_name
0,E12000001,100000.0,households_low_income_no_savings
1,E12000002,400000.0,households_low_income_no_savings
2,E12000003,300000.0,households_low_income_no_savings


In [11]:
 # Index of Multiple deprivation
data = pd.read_csv(f'{DATA_DIR}/imd/imd.csv')
data  = data[data.variable_name == 'Average score'] #adding this as in future there will be measures for each age cat.#
imd = data.loc[data.dataset == 'IMD'].copy()
imd['variable_name'] = 'imd_average_score'

imd_older_people = data.loc[data.dataset == 'IDAOPI'].copy()
imd_older_people['variable_name'] = 'imd_older_people'

imd_children = data.loc[data.dataset == "IDACI"].copy()
imd_children['variable_name'] = 'imd_children'


In [12]:
labour_market = pd.read_csv(f'{DATA_DIR}/labour-market/labour-market.csv')
unemployment = labour_market[labour_market.variable_name == "Unemployment rate - aged 16-64"]
unemployment = unemployment[unemployment.date == max(unemployment.date)]
unemployment['variable_name'] = 'unemployment_rate_16_64'

labour_market = pd.read_csv(f'{DATA_DIR}/labour-market/labour-market.csv')
economic_inactivity = labour_market[labour_market.variable_name == "% who are economically inactive - aged 16-64"]
economic_inactivity = economic_inactivity[economic_inactivity.date == max(economic_inactivity.date)]
economic_inactivity = economic_inactivity.replace("% who are economically inactive - aged 16-64","economic_inactivity_16_64")

economic_inactivity

,date,geography_code,geography_name,variable_code,variable_name,value
16850,2022-12-01,E06000005,Darlington,111,economic_inactivity_16_64,20.9
16853,2022-12-01,E06000047,County Durham,111,economic_inactivity_16_64,24.3
16856,2022-12-01,E06000001,Hartlepool,111,economic_inactivity_16_64,27.3
16859,2022-12-01,E06000002,Middlesbrough,111,economic_inactivity_16_64,29.3
16862,2022-12-01,E06000057,Northumberland,111,economic_inactivity_16_64,25.8
...,...,...,...,...,...,...
17069,2022-12-01,E10000017,Lancashire,111,economic_inactivity_16_64,23.5
17072,2022-12-01,E10000023,North Yorkshire,111,economic_inactivity_16_64,20.0
17075,2022-12-01,E12000001,North East,111,economic_inactivity_16_64,26.0
17078,2022-12-01,E12000002,North West,111,economic_inactivity_16_64,23.3


In [13]:
housing_benefit = pd.read_csv(f'{DATA_DIR}/HB/claimants.csv')
housing_benefit['Month'] = pd.to_datetime(housing_benefit['Month'])
housing_benefit.rename(columns={'Unnamed: 0': 'dt_idx'}, inplace=True)
housing_benefit = housing_benefit.loc[housing_benefit.groupby('geography_code')['dt_idx'].idxmax()]
housing_benefit[housing_benefit.geography_code.str.startswith('E08')]

,dt_idx,Month,geography_code,value,variable_name
1651,4896,2023-02-01,E08000001,12115.0,HB_claimants
1710,5073,2023-02-01,E08000002,6472.0,HB_claimants
1769,5250,2023-02-01,E08000003,30631.0,HB_claimants
1828,5427,2023-02-01,E08000004,8589.0,HB_claimants
1887,5604,2023-02-01,E08000005,10194.0,HB_claimants
1946,5781,2023-02-01,E08000006,14051.0,HB_claimants
2005,5958,2023-02-01,E08000007,8977.0,HB_claimants
2064,6135,2023-02-01,E08000008,10009.0,HB_claimants
2123,6312,2023-02-01,E08000009,6830.0,HB_claimants
2182,6489,2023-02-01,E08000010,11410.0,HB_claimants


In [14]:
smi = pd.read_csv(f'{DATA_DIR}/smi/smi_loans_in_payment_households.csv')
smi['Quarter'] = pd.to_datetime(smi['Quarter'])
smi.rename(columns={'Unnamed: 0': 'dt_idx'}, inplace=True)
smi = smi.loc[smi.groupby('geography_code')['Quarter'].idxmax()]
smi

,dt_idx,Quarter,geography_code,value,variable_name
9,9,2023-02-01,E12000001,537.0,smi_loans_in_payment_households
29,29,2023-02-01,E12000002,1752.0,smi_loans_in_payment_households
49,49,2023-02-01,E12000003,963.0,smi_loans_in_payment_households


In [15]:
homelessness = pd.read_csv(f'{DATA_DIR}/statutory-homelessness/statutory-homelessness.csv')
homelessness['date'] = homelessness['date'].str[:4]
homelessness['date'] = pd.to_numeric(homelessness['date'])
homelessness = homelessness.loc[homelessness.groupby('geography_code')['date'].idxmax()]
homelessness

,date,geography_code,geography_name,variable_name,value
129,2023,E06000001,Hartlepool,Total households assessed as owed a duty,174.0
174,2023,E06000002,Middlesbrough,Total households assessed as owed a duty,382.0
208,2023,E06000003,Redcar & Cleveland,Total households assessed as owed a duty,103.0
260,2023,E06000004,Stockton-on-Tees,Total households assessed as owed a duty,838.0
79,2023,E06000005,Darlington,Total households assessed as owed a duty,321.0
...,...,...,...,...,...
7,2023,E12000006,East of England,Total households assessed as owed a duty,7640.0
1,2023,E12000007,London,Total households assessed as owed a duty,14320.0
9,2023,E12000008,South East,Total households assessed as owed a duty,10930.0
10,2023,E12000009,South West,Total households assessed as owed a duty,7520.0


In [45]:
rental_prices = pd.read_csv(f'{DATA_DIR}/rental-prices/rental-prices.csv')
rental_prices['date'] = rental_prices['date'].str[:4]
rental_prices['date'] = pd.to_numeric(rental_prices['date'])
rental_prices = rental_prices[rental_prices.variable_code == 'Mean']
rental_prices = rental_prices.loc[rental_prices.groupby(['geography_code', 'property_name'])['date'].idxmax()]
#print(rental_prices.variable_code.unique())
rental_prices_pivot = rental_prices.pivot(index='property_code', columns='geography_code', values='value')
rental_prices_pivot.to_csv(os.path.join(SRC_DATA_DIR, 'bar_charts/current_rental_prices.csv'))

In [40]:
house_prices = pd.read_csv(f'{DATA_DIR}/house-prices/house-prices.csv')
house_prices['date'] = pd.to_datetime(house_prices['date'])
house_prices = house_prices.loc[house_prices.groupby(['geography_code'])['date'].idxmax()]
print(len(house_prices.geography_code.unique()))
geo = pd.read_csv(f"{DATA_DIR}/geo/geography_lookup.csv")

tot = len(geo.LAD22CD.unique()) + len(geo.WD22CD.unique())


1314


Concatenate the loaded data, then pivot into a table with a line per geography code. Filter this to only include geographies that are in the canonical list of areas.

In [18]:
place_data = pd.concat([
    population_data,
    area_of_place,
    gva,
    households,
    council_tax_data,
    clif_data,
    number_of_children,
    savings,
    imd,
    house_prices,
    households_in_poverty,
    unemployment,
    economic_inactivity,
    imd_children,
    imd_older_people,
    housing_benefit,
    smi,
    homelessness
]).pivot(index='geography_code', columns='variable_name', values='value')
place_data = place_data.loc[place_data.index.isin(get_place_list())]
place_data = place_data.merge(get_place_data(), left_index=True, right_index=True, how='outer')
place_data.index.name = 'geography_code'
place_data

,Area in sq km,GVA,HB_claimants,Households in poverty,Median house price,Number of households,Number of persons,Total households assessed as owed a duty,children_in_low_income,council_tax_pensioners,...,imd_children,imd_older_people,number_of_children,smi_loans_in_payment_households,unemployment_rate_16_64,ancestors,parents,children,name,type
geography_code,,,,,,,,,,,,,,,,,,,,,
E05000650,6.556496,82.047782,374.0,NaN,196000.0,5980.0,14133.0,NaN,NaN,NaN,...,NaN,NaN,3390.0,NaN,NaN,"[E08000001, E47000001, E12000002, E12999901]",[E08000001],[],Astley Bridge,WD22
E05000651,8.998946,79.731727,251.0,NaN,250000.0,5300.0,11331.0,NaN,NaN,NaN,...,NaN,NaN,2530.0,NaN,NaN,"[E08000001, E47000001, E12000002, E12999901]",[E08000001],[],Bradshaw,WD22
E05000652,3.719582,178.319099,860.0,NaN,169975.0,6810.0,14078.0,NaN,NaN,NaN,...,NaN,NaN,3722.0,NaN,NaN,"[E08000001, E47000001, E12000002, E12999901]",[E08000001],[],Breightmet,WD22
E05000653,7.340085,110.950950,175.0,NaN,253000.0,5680.0,13503.0,NaN,NaN,NaN,...,NaN,NaN,2813.0,NaN,NaN,"[E08000001, E47000001, E12000002, E12999901]",[E08000001],[],Bromley Cross,WD22
E05000654,3.497711,275.878141,NaN,NaN,120000.0,6360.0,16828.0,NaN,NaN,NaN,...,NaN,NaN,4875.0,NaN,NaN,"[E08000001, E47000001, E12000002, E12999901]",[E08000001],[],Crompton,WD22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E47000003,2023.378010,60136.000000,NaN,NaN,NaN,1035190.0,2349987.0,NaN,NaN,NaN,...,NaN,NaN,586078.0,NaN,NaN,"[E12000003, E12999901]",[E12000003],"[E08000032, E08000033, E08000034, E08000035, E...",West Yorkshire,CAUTH22
E47000004,723.925858,35345.000000,NaN,NaN,NaN,729960.0,1551722.0,NaN,NaN,NaN,...,NaN,NaN,346827.0,NaN,NaN,"[E12000002, E12999901]","[E12000002, E12000002]","[E08000011, E08000012, E08000013, E08000014, E...",Liverpool City Region,CAUTH22
E47000006,792.415536,14240.000000,NaN,NaN,NaN,317080.0,678173.0,NaN,NaN,NaN,...,NaN,NaN,159599.0,NaN,NaN,"[E12000001, E12999901]",[E12000001],"[E06000001, E06000002, E06000003, E06000004, E...",Tees Valley,CAUTH22


Create some additional metrics based on the data in place_data

In [19]:
place_data['Population density'] = place_data['Number of persons'] / place_data['Area in sq km']

Finally, write the data to an interim parquet and json file for later usage.

In [20]:
os.makedirs(f'{DATA_DIR}/interim/', exist_ok=True)
place_data.to_parquet(f'{DATA_DIR}/interim/place_data.parquet')
place_data.reset_index().to_json(f"{DATA_DIR}/interim/place_data.json", orient="records")

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [ ]:
house_prices = pd.read_csv(f'{DATA_DIR}/house-prices/house-prices.csv')
house_prices['date'] = pd.to_datetime(house_prices['date'])
#house_prices = house_prices.loc[house_prices.groupby(['geography_code'])['date'].idxmax()]
house_prices = house_prices.pivot(index='date', columns="geography_code", values="value")
house_prices.to_csv(os.path.join(SRC_DATA_DIR, 'time_series/house_prices.csv'))
house_prices.max()

geography_code
E05000650    196000.0
E05000651    250000.0
E05000652    169975.0
E05000653    253000.0
E05000654    120000.0
               ...   
E11000006    187000.0
E11000007    165000.0
E12000001    158000.0
E12000002    197000.0
E12000003    190000.0
Length: 1314, dtype: float64